# Testing Charter Locally

This notebook demonstrates how to test the charter package locally.

## Setup

First, make sure the package is installed in development mode:

```bash
# From the project root directory
pip install -e .
# or with dev dependencies
pip install -e ".[dev]"
```

If you're using a Jupyter kernel in a virtual environment, make sure the kernel is using the same environment where you installed the package.


In [ ]:
# Import charter
from charter import from_dict, from_json, from_file
import json

print("✓ Charter imported successfully")


✓ Schemantic imported successfully


## Basic Usage - Simple Schema


In [2]:
# Define a simple JSON schema
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "integer"},
        "email": {"type": "string"}
    },
    "required": ["name", "age"]
}

# Generate a Pydantic model
Person = from_dict(schema, "Person")

# Create an instance
person = Person(name="Alice", age=30, email="alice@example.com")
print(f"✓ Created person: {person.name}, age {person.age}")


✓ Created person: Alice, age 30


## Standard JSON Schema Keywords


In [3]:
# Test standard JSON Schema keywords
schema = {
    "type": "object",
    "properties": {
        "code": {
            "type": "string",
            "minLength": 3,
            "maxLength": 10,
            "pattern": "^[a-z]+$"
        },
        "status": {
            "type": "string",
            "enum": ["active", "inactive", "pending"]
        },
        "score": {
            "type": "number",
            "minimum": 0,
            "maximum": 100
        }
    },
    "required": ["code", "status"]
}

Model = from_dict(schema, "TestModel")

# Valid instance
instance = Model(code="abc", status="active", score=50)
print(f"✓ Valid instance: code={instance.code}, status={instance.status}, score={instance.score}")

# Test validation (this should raise an error)
try:
    invalid = Model(code="ab", status="active", score=50)  # Too short
except Exception as e:
    print(f"✓ Validation caught error: {type(e).__name__}")


✓ Valid instance: code=abc, status=active, score=50.0
✓ Validation caught error: ValidationError


## Nested Objects


In [4]:
# Test nested objects
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "address": {
            "type": "object",
            "properties": {
                "street": {"type": "string"},
                "city": {"type": "string"},
                "zipcode": {"type": "string"}
            },
            "required": ["street", "city"]
        }
    },
    "required": ["name", "address"]
}

Person = from_dict(schema, "Person")

person = Person(
    name="Alice",
    address={
        "street": "123 Main St",
        "city": "New York",
        "zipcode": "10001"
    }
)

print(f"✓ Nested object: {person.name} lives at {person.address.street}, {person.address.city}")


✓ Nested object: Alice lives at 123 Main St, New York


## Coercion and Validation (Charter Extensions)


In [5]:
# Test coercion and validation
schema = {
    "type": "object",
    "properties": {
        "flight_number": {
            "type": "integer",
            "coercion": "coerce_to_integer"  # Convert string/float to int
        },
        "destination": {
            "type": "string",
            "coercion": "coerce_to_string",
            "validations": {
                "min_length": {"threshold": 3},
                "max_length": {"threshold": 3},
                "no_capital_characters": None,
                "only_allow": {"allowed_values": ["abc", "def", "ghi"]}
            }
        },
        "distance": {
            "type": "number",
            "coercion": "coerce_to_float",
            "validations": {
                "greater_than_or_equal_to": {"threshold": 0}
            }
        }
    },
    "required": ["flight_number", "destination", "distance"]
}

Flight = from_dict(schema, "Flight")

# Coercion happens automatically
flight = Flight(
    flight_number="123",    # Coerced to int: 123
    destination="abc",      # Passes all validations
    distance="100.5"        # Coerced to float: 100.5
)

print(f"✓ Coercion: flight_number={flight.flight_number} (was string), distance={flight.distance} (was string)")
print(f"✓ Validation: destination={flight.destination} passed all checks")


✓ Coercion: flight_number=123 (was string), distance=100.5 (was string)
✓ Validation: destination=abc passed all checks


## Verify Installation

Check that the package is installed correctly and in editable mode:


In [ ]:
# Verify installation
import charter
import os

print(f"Version: {charter.__version__}")
print(f"Package location: {os.path.dirname(charter.__file__)}")
print(f"Is editable install: {'site-packages' not in charter.__file__}")
